# Data Cleaning and Preparation of Career Village datasets

## Load libraries

In [1]:
#run both R and python 
%load_ext rpy2.ipython

In [2]:
#mount google drive to load pre installs and zipped libraries
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
#extract library zip file "cv.library.tar.gz" from location - https://drive.google.com/drive/folders/1z3tM37Cf7zr3hXF5EtPMRnMwdoEb6arm
#make sure the folder is added to your personal google drive
%cp drive/MyDrive/R_libraries/cv.library.tar.gz .

In [4]:
!du -sh cv.library.tar.gz

35M	cv.library.tar.gz


In [5]:
#copy library to current colab directory
!tar xf cv.library.tar.gz

tar: Removing leading `/' from member names


In [6]:
%ls usr/local/lib/R/site-library/

crosstalk/    IRdisplay/  later/     lubridate/  plotly/    Rcpp/  tidyverse/
htmlwidgets/  IRkernel/   lazyeval/  pbdZMQ/     promises/  repr/


In [7]:
%%R
.libPaths('usr/local/lib/R/site-library/')
.libPaths()

[1] "/content/usr/local/lib/R/site-library"
[2] "/usr/local/lib/R/site-library"        
[3] "/usr/lib/R/site-library"              
[4] "/usr/lib/R/library"                   


In [8]:
#install python libraries
from IPython.display import display, HTML

In [9]:
#load R libraries
#use %%R for every cell that runs R code
%%R
library(tidyverse)
library(dplyr)
library(plotly)
library(lubridate)
library(ggplot2)
library(readr)

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

R[write to console]: 
Attaching package: ‘plotly’


R[write to console]: The following object is masked from ‘package:ggplot2’:

    last_plot


R[write to console]: The following object is masked from ‘package:stats’:

    filter


R[write to console]: The following object is masked from ‘package:graphics’:

    layout


R[write to console]: 
Attaching package: ‘lubridate’


R[write to console]: The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [10]:
%%R
url <- 'https://media.githubusercontent.com/media/nthammadi-uncc/CareerVillage/main/data/raw/'
all_data_files <- list('answer_scores','answers','comments','emails','group_memberships','groups','matches','professionals','question_scores','questions','school_memberships','students','tag_questions','tag_users','tags')  
#dynamially assign dataframe names for all the csv files
for(i in 1:length(all_data_files)) {                              
  assign(paste0(all_data_files[i], "_df"),                                  
         read.csv(url(paste0(url,
                   all_data_files[i],'.csv'))))
}


In [11]:
%%R
#view all dataframes created
ls()

 [1] "all_data_files"        "answer_scores_df"      "answers_df"           
 [4] "comments_df"           "emails_df"             "group_memberships_df" 
 [7] "groups_df"             "i"                     "matches_df"           
[10] "professionals_df"      "question_scores_df"    "questions_df"         
[13] "school_memberships_df" "students_df"           "tag_questions_df"     
[16] "tag_users_df"          "tags_df"               "url"                  


In [12]:
#use this cell to convert any R dataframe to python dataframe
#%R -o df - uncomment this line and specify the "df"

## Exploratory Data Analysis

### Exploring questions, questions_scores, tags, tag_questions, tag_users datasets

In [13]:
#convert R datafram to python dataframe
%R -o questions_df 
display(HTML(questions_df.head(5).to_html()))

,questions_id,questions_author_id,questions_date_added,questions_title,questions_body
1,332a511f1569444485cf7a7a556a5e54,8f6f374ffd834d258ab69d376dd998f5,2016-04-26 11:14:26 UTC+0000,Teacher career question,What is a maths teacher? what is a maths teacher useful? #college #professor #lecture
2,eb80205482e4424cad8f16bc25aa2d9c,acccbda28edd4362ab03fb8b6fd2d67b,2016-05-20 16:48:25 UTC+0000,I want to become an army officer. What can I do to become an army officer?,I am Priyanka from Bangalore . Now am in 10th std . When I go to college I should not get confused on what I want to take to become army officer. So I am asking this question #military #army
3,4ec31632938a40b98909416bdd0decff,f2c179a563024ccc927399ce529094b5,2017-02-08 19:13:38 UTC+0000,Will going abroad for your first job increase your chances for jobs back home?,"I'm planning on going abroad for my first job. It will be a teaching job and I don't have any serious career ideas. I don't know what job I would be working if I stay home instead so I'm assuming staying or leaving won't makeba huge difference in what I care about, unless I find something before my first job. I can think of ways that going abroad can be seen as good and bad. I do not know which side respectable employers willl side with. #working-abroad #employment- #overseas"
4,2f6a9a99d9b24e5baa50d40d0ba50a75,2c30ffba444e40eabb4583b55233a5a4,2017-09-01 14:05:32 UTC+0000,"To become a specialist in business management, will I have to network myself?",i hear business management is a hard way to get a job if you're not known in the right areas. #business #networking
5,5af8880460c141dbb02971a1a8369529,aa9eb1a2ab184ebbb00dc01ab663428a,2017-09-01 02:36:54 UTC+0000,Are there any scholarships out there for students that are first generation and live in GA?,I'm trying to find scholarships for first year students but they all seem to be for other states besides GA. Any help??\n\n#college\n#scholarships \n#highschoolsenior \n#firstgeneration \n


In [14]:
#convert R datafram to python dataframe
%R -o question_scores_df 
display(HTML(question_scores_df.head(5).to_html()))

,id,score
1,38436aadef3d4b608ad089cf53ab0fe7,5
2,edb8c179c5d64c9cb812a59a32045f55,4
3,333464d7484b43e3866e86096bc4ddb9,6
4,4b995e60b99d4ee18346e893e007cb8f,6
5,f6b9ca94aed04ba28256492708e74f60,6


In [15]:
#convert R datafram to python dataframe
%R -o tags_df 
display(HTML(tags_df.head(5).to_html()))

,tags_tag_id,tags_tag_name
1,27490,college
2,461,computer-science
3,593,computer-software
4,27292,business
5,18217,doctor


In [16]:
#convert R datafram to python dataframe
%R -o tag_questions_df 
display(HTML(tag_questions_df.head(5).to_html()))

,tag_questions_tag_id,tag_questions_question_id
1,28930,cb43ebee01364c68ac61d347a393ae39
2,28930,47f55e85ce944242a5a347ab85a8ffb4
3,28930,ccc30a033a0f4dfdb2eb987012f25792
4,28930,e30b274e48d741f7bf50eb5e7171a3c0
5,28930,3d22742052df4989b311b4195cbb0f1a


In [17]:
#convert R datafram to python dataframe
%R -o tag_users_df 
display(HTML(tag_users_df.head(5).to_html()))

,tag_users_tag_id,tag_users_user_id
1,593,c72ab38e073246e88da7e9a4ec7a4472
2,1642,8db519781ec24f2e8bdc67c2ac53f614
3,638,042d2184ee3e4e548fc3589baaa69caf
4,11093,c660bd0dc1b34224be78a58aa5a84a63
5,21539,8ce1dca4e94240239e4385ed22ef43ce


#### Working with tag_questions, tags, tag_users datasets

In [18]:
%%R
#join tags with tag_question, to get the most popular tags 
tag_ques_df <- left_join(tags_df, tag_questions_df, by = c("tags_tag_id"="tag_questions_tag_id"))   
print(as_tibble(tag_ques_df))

# A tibble: 85,731 × 3
   tags_tag_id tags_tag_name tag_questions_question_id       
         <int> <chr>         <chr>                           
 1       27490 college       bfbf79503bfb48909b4e514c5675502e
 2       27490 college       a4da117f4e3b45d49b83f7c3d6bc9915
 3       27490 college       9b90fda0660e4b72936c17a642334929
 4       27490 college       37a7d709c90445fc9643ada3933fbc6d
 5       27490 college       bc854d4070dd474797ff5f0242f510f1
 6       27490 college       b3f8dc8bb99b45c6a6898b7ea20daf6c
 7       27490 college       2fceaae061a745728d59c858670ad095
 8       27490 college       a13642cafac5475c8ba7012e13e06b28
 9       27490 college       4c521ed278a3450b995d8218aeae2da9
10       27490 college       81fe26c0f4214d70b1c0875f3a4720a0
# … with 85,721 more rows


In [19]:
%%R
#group by tag_id to get count of questions associated to that tag
tag_ques_df <- tag_ques_df %>% group_by(tags_tag_id,tags_tag_name) %>%
                  summarise(question_count = n_distinct(tag_questions_question_id,na.rm = TRUE))
print(as_tibble(tag_ques_df))

`summarise()` has grouped output by 'tags_tag_id'. You can override using the
`.groups` argument.
# A tibble: 16,269 × 3
   tags_tag_id tags_tag_name question_count
         <int> <chr>                  <int>
 1          27 military                 134
 2          29 teaching                 520
 3          36 health                   214
 4          42 veterinary               173
 5          46 lawyer                   285
 6          51 biology                  464
 7          53 science                  701
 8          54 engineering             1083
 9          55 engineer                 505
10          60 google                    21
# … with 16,259 more rows


In [20]:
%%R
#join tags with tag_users that has professional id
prof_tags_df <- left_join(tag_users_df, tags_df, by = c("tag_users_tag_id"="tags_tag_id"))  
print(as_tibble(prof_tags_df))

# A tibble: 136,663 × 3
   tag_users_tag_id tag_users_user_id                tags_tag_name    
              <int> <chr>                            <chr>            
 1              593 c72ab38e073246e88da7e9a4ec7a4472 computer-software
 2             1642 8db519781ec24f2e8bdc67c2ac53f614 programming      
 3              638 042d2184ee3e4e548fc3589baaa69caf running          
 4            11093 c660bd0dc1b34224be78a58aa5a84a63 life-coach       
 5            21539 8ce1dca4e94240239e4385ed22ef43ce art              
 6             1047 3330f8a7835346a2a91f9393ae21efee snowboarding     
 7               64 5a4a16842ec64430ac3f916aacf35fe1 architecture     
 8             1139 461f92b955924604832a92b6bc14ac1d gardening        
 9               55 7daf1e6dfb3443b99b240890f0a4d69b engineer         
10               54 7daf1e6dfb3443b99b240890f0a4d69b engineering      
# … with 136,653 more rows


In [21]:
%%R
#group by tag id and tag name to get all the professionals associated to that tag
tag_prof_count_df <- prof_tags_df %>% group_by(tag_users_tag_id, tags_tag_name) %>%
                  summarise(all_users = paste(tag_users_user_id, collapse = " | "), professionals_count = n_distinct(tag_users_user_id,na.rm = TRUE))
print(as_tibble(tag_prof_count_df))

`summarise()` has grouped output by 'tag_users_tag_id'. You can override using
the `.groups` argument.
# A tibble: 11,911 × 4
   tag_users_tag_id tags_tag_name all_users                     professionals_c…
              <int> <chr>         <chr>                                    <int>
 1               27 military      5f77e0a2c3a144dda336df2294a6…              210
 2               29 teaching      6e795d8d47de4676a8d5cef449be…              359
 3               36 health        0c9a2748560541be9fe2df0d7be8…              141
 4               42 veterinary    9446cf87650141ac92b037d051a0…               94
 5               46 lawyer        b6af2f682f0e4bf2af7783c19808…              189
 6               51 biology       206f648e7e88405c858c0e6b4a35…              272
 7               53 science       91b249c2b66e44c7ac2c9bd2a725…              396
 8               54 engineering   7daf1e6dfb3443b99b240890f0a4…              742
 9               55 engineer      7daf1e6dfb3443b99b240890f0a4… 

In [22]:
%%R
#join tag_ques_df with tag_prof_count_df, to get the most popular tags with professionals
tag_ques_prof_df <- left_join(tag_ques_df, select(tag_prof_count_df,c(tag_users_tag_id,professionals_count)), by = c("tags_tag_id"="tag_users_tag_id"))   
print(as_tibble(tag_ques_prof_df))

# A tibble: 16,269 × 4
   tags_tag_id tags_tag_name question_count professionals_count
         <int> <chr>                  <int>               <int>
 1          27 military                 134                 210
 2          29 teaching                 520                 359
 3          36 health                   214                 141
 4          42 veterinary               173                  94
 5          46 lawyer                   285                 189
 6          51 biology                  464                 272
 7          53 science                  701                 396
 8          54 engineering             1083                 742
 9          55 engineer                 505                 168
10          60 google                    21                  47
# … with 16,259 more rows


In [23]:
%%R
#join tag_question with tags 
ques_tags_df <- left_join(tag_questions_df, tags_df, by = c("tag_questions_tag_id"="tags_tag_id"))   
print(as_tibble(ques_tags_df))

# A tibble: 76,553 × 3
   tag_questions_tag_id tag_questions_question_id        tags_tag_name
                  <int> <chr>                            <chr>        
 1                28930 cb43ebee01364c68ac61d347a393ae39 minor        
 2                28930 47f55e85ce944242a5a347ab85a8ffb4 minor        
 3                28930 ccc30a033a0f4dfdb2eb987012f25792 minor        
 4                28930 e30b274e48d741f7bf50eb5e7171a3c0 minor        
 5                28930 3d22742052df4989b311b4195cbb0f1a minor        
 6                28930 c79baebeb6d44726b6f70a2414fb69bc minor        
 7                28930 4eb64ed1527847f29d3e3d8a713bb3dc minor        
 8                28930 bebfb741f4524b8cb871f35b5780b54f minor        
 9                28930 3e7c462a0adb4bfa8830282867f3900b minor        
10                28930 23b52dd81ad7465a9472596c7dedc492 minor        
# … with 76,543 more rows


In [24]:
%%R
print(dim(tag_questions_df))
print(dim(tags_df))
print(dim(ques_tags_df))

[1] 76553     2
[1] 16269     2
[1] 76553     3


In [25]:
%%R
#group by question_id to get all the tags associated to that question
ques_tags_df <- ques_tags_df %>% group_by(tag_questions_question_id) %>%
                  summarise(all_tags = paste(tags_tag_name, collapse = " | "), tag_count = n_distinct(tag_questions_tag_id,na.rm = TRUE))
print(as_tibble(ques_tags_df))

# A tibble: 23,288 × 3
   tag_questions_question_id        all_tags                           tag_count
   <chr>                            <chr>                                  <int>
 1 0003e7bf48f24b5c985f8fce96e611f3 internship | technology | high-sc…         6
 2 0006609dd4da40dcaa5a83e0499aba14 psychology | law                           2
 3 000af224bc2f4e94a19f8b62ba279cc4 biology | marine                           2
 4 000b30fb534b41f7b716fa9ebf9c3f35 teaching | exercise-science | sch…         4
 5 0018752e44b44e26bb74a0a43232b4d6 math | puremathematics                     2
 6 0025b917219c4a24ba5fe260ca66ca98 forensic | criminal-justice | col…         3
 7 00269cdc24fd4a3c93aad485688c240b videogames | videogamesmaking | g…         3
 8 002b07f6281a407ca49ee8b3538e5e45 programming                                1
 9 002c5689a0b642a3940f99b68f5364b6 clinical-psychology | psychiatry …         4
10 0031077b31454fadae1754e871e84457 interior-decorator | interior-des…         2
# … w

In [26]:
%%R 
# remove duplicate tags from ques_tags_df
for (i in 1:nrow(ques_tags_df)) {
    #get the tags cell value
    all_tags <- c(ques_tags_df[i,2])
    #if there are more than one tag associated to the questiom,
    #only keep the unique tags
    if(grepl("|", all_tags, fixed = TRUE)){
      tag_list <- unlist(strsplit(as.character(all_tags), " \\| "))
      tag_set <- unique(tag_list)
      ques_tags_df[i,2] <- paste(tag_set, collapse = " | ")
    }
}

In [27]:
%%R
print(dim(tags_df))
print(dim(tag_users_df))
print(dim(prof_tags_df))

[1] 16269     2
[1] 136663      2
[1] 136663      3


In [28]:
%%R
#group by professional id to get all the tags that are associated to the professional
prof_all_tags_df <- prof_tags_df %>% group_by(tag_users_user_id) %>%
                  summarise(all_tags = paste(tags_tag_name, collapse = " | "), tag_count = n_distinct(tag_users_tag_id,na.rm = TRUE))
print(as_tibble(prof_all_tags_df))

# A tibble: 30,202 × 3
   tag_users_user_id                all_tags                           tag_count
   <chr>                            <chr>                                  <int>
 1 00009a0f9bda43eba47104e9ac62aff5 digital-media | script-writing | …         3
 2 000196ef8db54b9a86ae70ad31745d04 accounting                                 1
 3 0008138be908438e8944b21f7f57f2c1 real-estate                                1
 4 000d4635e5da41e3bfd83677ee11dda4 university | information-technolo…         3
 5 000e2b5714444d79a672bf927905135c financial-services                         1
 6 0018873fbf7742aba1bf13fff12cbfa4 financial-services                         1
 7 001bd6f7f1ac4897b0b35dc665c64d2e college                                    1
 8 001df2a2cfb74ef78f3053ddee305ba6 food | family-medicine | creative…         9
 9 00211ce6a146458b849283adf31ffc13 telecommunications | #math | #spo…         3
10 00270bae68ee49eba99cb63a2942b5cd technology | testing | customer-a…        16
# … w

In [29]:
%%R
# remove duplicate tags from prof_all_tags_df
for (i in 1:nrow(prof_all_tags_df)) {
    #get the tags cell value
    all_tags <- c(prof_all_tags_df[i,2])
    #if there are more than one tag associated to the professional,
    #only keep the unique tags
    if(grepl("|", all_tags, fixed = TRUE)){
      tag_list <- unlist(strsplit(as.character(all_tags), " \\| "))
      tag_set <- unique(tag_list)
      prof_all_tags_df[i,2] <- paste(tag_set, collapse = " | ")
    }
}

#### Working with questions and question_scores datasets

In [30]:
%%R
#find duplicates in question id in questions and questions_score
print(as_tibble(questions_df[duplicated(questions_df$questions_id), ]))
print(as_tibble(question_scores_df[duplicated(question_scores_df$id), ]))

#this is not necessary, but just an additional check to find duplicates in ques_tags_df
print(as_tibble(ques_tags_df[duplicated(ques_tags_df$tag_questions_question_id), ]))

# A tibble: 0 × 5
# … with 5 variables: questions_id <chr>, questions_author_id <chr>,
#   questions_date_added <chr>, questions_title <chr>, questions_body <chr>
# A tibble: 0 × 2
# … with 2 variables: id <chr>, score <int>
# A tibble: 0 × 3
# … with 3 variables: tag_questions_question_id <chr>, all_tags <chr>,
#   tag_count <int>


In [31]:
%%R
print(dim(questions_df))
print(dim(question_scores_df))
print(dim(ques_tags_df))

[1] 23931     5
[1] 23928     2
[1] 23288     3


In [32]:
%%R
ques_anti_join_df <- anti_join(question_scores_df, questions_df, by = c("id"="questions_id"))
print(as_tibble(ques_anti_join_df)) 
ques_anti_join2_df <- anti_join(questions_df, question_scores_df, by = c("questions_id"="id"))
print(as_tibble(ques_anti_join2_df)) 

# A tibble: 0 × 2
# … with 2 variables: id <chr>, score <int>
# A tibble: 3 × 5
  questions_id  questions_autho… questions_date_… questions_title questions_body
  <chr>         <chr>            <chr>            <chr>           <chr>         
1 215a6e8eeda8… f511faad4fb14b4… 2018-03-12 03:2… Network Admini… "What is the …
2 0a1d9fec5b1f… f511faad4fb14b4… 2018-05-08 13:0… Interniship Te… "Is it possib…
3 486d7c131c69… f511faad4fb14b4… 2018-03-12 03:2… Remote Network… "Are you able…


Relationship between Questions and Question scores is one to one and 3 questions do not have scores

In [33]:
%%R
#join questions with questions_scores
ques_score_tag_df <- left_join(questions_df, question_scores_df, by = c("questions_id"="id"))  
#join questions with ques_tags_df
ques_score_tag_df <- left_join(ques_score_tag_df, ques_tags_df, by = c("questions_id"="tag_questions_question_id"))
print(as_tibble(ques_score_tag_df))

# A tibble: 23,931 × 8
   questions_id questions_autho… questions_date_… questions_title questions_body
   <chr>        <chr>            <chr>            <chr>           <chr>         
 1 332a511f156… 8f6f374ffd834d2… 2016-04-26 11:1… Teacher   care… "What  is  a …
 2 eb80205482e… acccbda28edd436… 2016-05-20 16:4… I want to beco… "I am Priyank…
 3 4ec31632938… f2c179a563024cc… 2017-02-08 19:1… Will going abr… "I'm planning…
 4 2f6a9a99d9b… 2c30ffba444e40e… 2017-09-01 14:0… To become a sp… "i hear busin…
 5 5af8880460c… aa9eb1a2ab184eb… 2017-09-01 02:3… Are there any … "I'm trying t…
 6 7c336403258… d1e4587c0e784c6… 2017-03-01 04:2… How many years… "To be an eng…
 7 be3c5edfdb0… 71b4554d4a82425… 2017-09-01 04:5… I want to beco… "I am a music…
 8 0f1d6a4f276… 585ac233015447c… 2016-05-19 22:1… what kind of  … "I like socce…
 9 d4999cdc470… 654e1d6fd5b9472… 2017-08-31 19:2… What are the c… "I'm asking b…
10 e214acfbe66… 16908136951a48e… 2012-09-09 05:3… what does it t… "I am a sopho…
# … w

### Exploring professionals, groups, group_memberships and school_memberships datasets

In [34]:
#convert R datafram to python dataframe
%R -o professionals_df 
display(HTML(professionals_df.head(5).to_html()))

,professionals_id,professionals_location,professionals_industry,professional_industry_group,professionals_headline,professionals_date_joined
1,9ced4ce7519049c0944147afb75a8ce3,,,,,2011-10-05 20:35:19 UTC+0000
2,f718dcf6d2ec4cb0a52a9db59d7f9e67,,,,,2011-10-05 20:49:21 UTC+0000
3,0c673e046d824ec0ad0ebe012a0673e4,"New York, New York",,,,2011-10-18 17:31:26 UTC+0000
4,977428d851b24183b223be0eb8619a8c,"Boston, Massachusetts",,,,2011-11-09 20:39:29 UTC+0000
5,e2d57e5041a44f489288397c9904c2b2,,,,,2011-12-10 22:14:44 UTC+0000


In [35]:
%%R
print(dim(professionals_df))
print(dim(prof_all_tags_df))

[1] 28152     6
[1] 30202     3


In [36]:
%%R
#find duplicates in professionals id in professionals and prof_all_tags_df
print(as_tibble(professionals_df[duplicated(professionals_df$professionals_id), ]))
print(as_tibble(prof_all_tags_df[duplicated(prof_all_tags_df$tag_users_user_id), ]))
#check which professionals are in the prof_all_tags_df but not in professionals_df
#anti_join_df <- anti_join(professionals_df, prof_all_tags_df, by = c("professionals_id"="tag_users_user_id"))
anti_join_df <- anti_join(prof_all_tags_df, professionals_df, by = c("tag_users_user_id"="professionals_id"))
summary(as_tibble(anti_join_df)) 

# A tibble: 0 × 6
# … with 6 variables: professionals_id <chr>, professionals_location <chr>,
#   professionals_industry <chr>, professional_industry_group <chr>,
#   professionals_headline <chr>, professionals_date_joined <chr>
# A tibble: 0 × 3
# … with 3 variables: tag_users_user_id <chr>, all_tags <chr>, tag_count <int>
 tag_users_user_id    all_tags           tag_count    
 Length:4608        Length:4608        Min.   : 1.00  
 Class :character   Class :character   1st Qu.: 2.00  
 Mode  :character   Mode  :character   Median : 3.00  
                                       Mean   : 4.14  
                                       3rd Qu.: 5.00  
                                       Max.   :56.00  


In [37]:
%%R
print(as_tibble(anti_join_df))

# A tibble: 4,608 × 3
   tag_users_user_id                all_tags                           tag_count
   <chr>                            <chr>                                  <int>
 1 001bd6f7f1ac4897b0b35dc665c64d2e college                                    1
 2 001df2a2cfb74ef78f3053ddee305ba6 food | family-medicine | creative…         9
 3 002f07c506a2438c9e7eb90693c3c2f2 real-estate | interior-design | f…         3
 4 003a1ef91d0a45f6a61ca56fffd1beee skincare | radio                           2
 5 003af75eba604db9a7b72862fd5efd03 english | law | politics | consti…         6
 6 004b221ad6534a0fac421356eea279b6 law | spanish | doublemajor | emi…         5
 7 0080e2394d3440c8b0efa71b4482b784 career | engineer | nanosystems            3
 8 0088fbc76ac54f059427b8d84827399b professor | student | counseling …         8
 9 008d073f619946ad80d2eac08be12020 teacher | professor | educator | …         5
10 0096ddada2924340be01d83357e0d229 career | career-choice | careers           3
# … wi

There can be some users in tag_users that do not have their records in professionals data. This can be a data error or data mismatch while data entry.
To make further analysis easier, we will only consider those ids that are present in professionals are populate the tags they are associated with

In [38]:
%%R
#join professionals with prof_all_tags_df
prof_all_tags_df <- left_join(professionals_df, prof_all_tags_df, by = c("professionals_id"="tag_users_user_id"))  
print(as_tibble(prof_all_tags_df))

# A tibble: 28,152 × 8
   professionals_id           professionals_l… professionals_i… professional_in…
   <chr>                      <chr>            <chr>            <chr>           
 1 9ced4ce7519049c0944147afb… ""               ""               ""              
 2 f718dcf6d2ec4cb0a52a9db59… ""               ""               ""              
 3 0c673e046d824ec0ad0ebe012… "New York, New … ""               ""              
 4 977428d851b24183b223be0eb… "Boston, Massac… ""               ""              
 5 e2d57e5041a44f489288397c9… ""               ""               ""              
 6 c9bfa93898594cbbace436dec… ""               ""               ""              
 7 ed85488fb5e941eaa97014137… ""               ""               ""              
 8 102fb92c28034ad988b593d01… ""               ""               ""              
 9 5a4a16842ec64430ac3f916aa… ""               ""               ""              
10 81999d5ad93549dab55636a54… ""               ""               ""              
# … w

In [39]:
#convert R datafram to python dataframe
%R -o groups_df 
display(HTML(groups_df.head(50).to_html()))

,groups_id,groups_group_type
1,eabbdf4029734c848a9da20779637d03,youth program
2,7080bf8dcf78463bb03e6863887fd715,youth program
3,bc6fc50a2b444efc8ec47111b290ffb8,youth program
4,37f002e8d5e442ca8e36e972eaa55882,youth program
5,52419ff84d4b47bebd0b0a6c1263c296,youth program
6,559dbc7bd1f64c268ff149c4d5d63500,youth program
7,528fa2c7559a40749b6151a07d3b8ef5,youth program
8,cfaac1762bed4c8bb88696267129a560,youth program
9,f79273f51df849d298bd6f4b86daee99,youth program
10,ba94226c87d249a1a2dbdc32a848b769,youth program


There are a lot of redundant group types

In [40]:
%%R
unique(groups_df$groups_group_type)

[1] "youth program"        "cause"                "professional network"
[4] "club"                 "competition"          "mentorship program"  
[7] "interest group"      


In [41]:
%%R
#find duplicates in group ids
print(as_tibble(groups_df[duplicated(groups_df$groups_id), ]))

# A tibble: 0 × 2
# … with 2 variables: groups_id <chr>, groups_group_type <chr>


In [42]:
#convert R datafram to python dataframe
%R -o group_memberships_df 
display(HTML(group_memberships_df.head().to_html()))

,group_memberships_group_id,group_memberships_user_id
1,eabbdf4029734c848a9da20779637d03,9a5aead62c344207b2624dba90985dc5
2,eabbdf4029734c848a9da20779637d03,ea7122da1c7b4244a2184a4f9f944053
3,eabbdf4029734c848a9da20779637d03,cba603f34acb4a40b3ccb53fe6681b5d
4,eabbdf4029734c848a9da20779637d03,fa9a126e63714641ae0145557a390cab
5,eabbdf4029734c848a9da20779637d03,299da113c5d1420ab525106c242c9429


In [43]:
%%R
#find duplicates in group ids and user ids
print(as_tibble(group_memberships_df[duplicated(group_memberships_df$group_memberships_group_id), ]))
print(as_tibble(group_memberships_df[duplicated(group_memberships_df$group_memberships_user_id), ]))

# A tibble: 992 × 2
   group_memberships_group_id       group_memberships_user_id       
   <chr>                            <chr>                           
 1 eabbdf4029734c848a9da20779637d03 ea7122da1c7b4244a2184a4f9f944053
 2 eabbdf4029734c848a9da20779637d03 cba603f34acb4a40b3ccb53fe6681b5d
 3 eabbdf4029734c848a9da20779637d03 fa9a126e63714641ae0145557a390cab
 4 eabbdf4029734c848a9da20779637d03 299da113c5d1420ab525106c242c9429
 5 7080bf8dcf78463bb03e6863887fd715 82cf96ae74fa4b3a8ffd8a74446c08ca
 6 7080bf8dcf78463bb03e6863887fd715 b0f6c44506444fb99e910dcc5836b5d8
 7 bc6fc50a2b444efc8ec47111b290ffb8 370dba85f183496186772be0c53a69b1
 8 bc6fc50a2b444efc8ec47111b290ffb8 ff165c6b122b47b9a5d4db2504086460
 9 bc6fc50a2b444efc8ec47111b290ffb8 4d7b8bf7e9c547a4a49c39114cd378e0
10 bc6fc50a2b444efc8ec47111b290ffb8 25b0211225e5436cacdfa360c47923eb
# … with 982 more rows
# A tibble: 311 × 2
   group_memberships_group_id       group_memberships_user_id       
   <chr>                            <chr

A professional can be assigned to multiple groups

In [44]:
%%R
print(dim(groups_df))
print(dim(group_memberships_df))

[1] 49  2
[1] 1038    2


In [45]:
%%R
#join group memberships and groups
group_memberships_df <- left_join(group_memberships_df, groups_df, by = c("group_memberships_group_id"="groups_id"))  
print(as_tibble(group_memberships_df))

# A tibble: 1,038 × 3
   group_memberships_group_id       group_memberships_user_id   groups_group_ty…
   <chr>                            <chr>                       <chr>           
 1 eabbdf4029734c848a9da20779637d03 9a5aead62c344207b2624dba90… youth program   
 2 eabbdf4029734c848a9da20779637d03 ea7122da1c7b4244a2184a4f9f… youth program   
 3 eabbdf4029734c848a9da20779637d03 cba603f34acb4a40b3ccb53fe6… youth program   
 4 eabbdf4029734c848a9da20779637d03 fa9a126e63714641ae0145557a… youth program   
 5 eabbdf4029734c848a9da20779637d03 299da113c5d1420ab525106c24… youth program   
 6 7080bf8dcf78463bb03e6863887fd715 836a747118d6436caf56ff3a3c… youth program   
 7 7080bf8dcf78463bb03e6863887fd715 82cf96ae74fa4b3a8ffd8a7444… youth program   
 8 7080bf8dcf78463bb03e6863887fd715 b0f6c44506444fb99e910dcc58… youth program   
 9 bc6fc50a2b444efc8ec47111b290ffb8 ab8d405cfdab4faf83ffe7f839… youth program   
10 bc6fc50a2b444efc8ec47111b290ffb8 370dba85f183496186772be0c5… youth program   
# … wi

In [46]:
%%R
#group professionals to get all their group membership details
prof_groups_df <- group_memberships_df %>% group_by(group_memberships_user_id) %>%
                  summarise(all_groups = paste(groups_group_type, collapse = " | "), group_count = n_distinct(group_memberships_group_id,na.rm = TRUE))
print(as_tibble(prof_groups_df))

# A tibble: 727 × 3
   group_memberships_user_id        all_groups                       group_count
   <chr>                            <chr>                                  <int>
 1 00a4e4e6d6654b7f81d4b9be65008dc4 youth program                              1
 2 018ed03f99f949739335a319e7e7027d youth program | youth program |…           6
 3 01d85b9badfb44f2954b5990c668556e cause                                      1
 4 020b27caf2514094a158c719311ea379 cause                                      1
 5 0271a27905b3425b85552f80a13618af youth program                              1
 6 02aab9809abc442bb9ee99c20c41e757 youth program | youth program |…           4
 7 02fd718863804d348f3afa7a569070c4 youth program | youth program              2
 8 03913cfd151f46a196786f5a21c87085 cause                                      1
 9 0419379c68ca4006b16f3b62638e2d34 professional network                       1
10 05ab77d4c6a141b999044ebbf5415b0d youth program | youth program              2
# … with

In [47]:
%%R 
# remove duplicate group types from prof_groups_df
for (i in 1:nrow(prof_groups_df)) {
    #get the groups cell value
    all_groups <- c(prof_groups_df[i,2])
    #if there are more than one group associated to the professional,
    #only keep the unique group types
    if(grepl("|", all_groups, fixed = TRUE)){
      group_list <- unlist(strsplit(as.character(all_groups), " \\| "))
      group_set <- unique(group_list)
      prof_groups_df[i,2] <- paste(group_set, collapse = " | ")
    }
}

In [48]:
#convert R datafram to python dataframe
%R -o prof_groups_df 
display(HTML(prof_groups_df.head().to_html()))

,group_memberships_user_id,all_groups,group_count
1,00a4e4e6d6654b7f81d4b9be65008dc4,youth program,1
2,018ed03f99f949739335a319e7e7027d,youth program | cause | professional network,6
3,01d85b9badfb44f2954b5990c668556e,cause,1
4,020b27caf2514094a158c719311ea379,cause,1
5,0271a27905b3425b85552f80a13618af,youth program,1


In [49]:
#convert R datafram to python dataframe
%R -o school_memberships_df 
display(HTML(school_memberships_df.head().to_html()))

,school_memberships_school_id,school_memberships_user_id
1,197406,23dce13ca6164a73aec7a3cd56a4884d
2,197398,23dce13ca6164a73aec7a3cd56a4884d
3,199821,23dce13ca6164a73aec7a3cd56a4884d
4,186239,9c5803ae43ca4cf6b27ea85871625116
5,182063,9c5803ae43ca4cf6b27ea85871625116


In [50]:
%%R
#find duplicates in school ids and user ids
print(as_tibble(school_memberships_df[duplicated(school_memberships_df$school_memberships_school_id), ]))
print(as_tibble(school_memberships_df[duplicated(school_memberships_df$school_memberships_user_id), ]))

# A tibble: 2,932 × 2
   school_memberships_school_id school_memberships_user_id      
                          <int> <chr>                           
 1                       196393 bc46e3699d92477ba8c7aa723e54a151
 2                       196401 17001ea532b044b08a006a989924e2e9
 3                       197406 f857f0fd5f8d4337a4ee3497298f695a
 4                       197007 557be8f56a604b75b885c208160ff0a9
 5                       196343 9e90c659a3e34be28c3b97695f1cf518
 6                       197246 96bbbdd06a334805a0501034d9df1aa4
 7                        77907 faa0fe0510804c4abeffc171acf37ba2
 8                       200218 85f378f43eee44c986addf5fc27038ce
 9                       200475 e1f680fc6bdc472dbe3024f7bf1aa290
10                       199987 f96e9335c8bf44cc9f018384af02f285
# … with 2,922 more rows
# A tibble: 2,457 × 2
   school_memberships_school_id school_memberships_user_id      
                          <int> <chr>                           
 1                   

A professional can be associated with multiple schools

In [51]:
%%R
#group professionals to get all their school membership details
prof_schools_df <- school_memberships_df %>% group_by(school_memberships_user_id) %>%
                  summarise(all_schools = paste(school_memberships_school_id, collapse = " | "), school_count = n_distinct(school_memberships_school_id,na.rm = TRUE))
print(as_tibble(prof_schools_df))

# A tibble: 3,181 × 3
   school_memberships_user_id       all_schools              school_count
   <chr>                            <chr>                           <int>
 1 001f3b39d8884a41ab7e71789b6a53a0 108854 | 197363 | 197278            3
 2 00458835327245d7bce9103ff114eceb 196670                              1
 3 00594d1a266f4907ac93a689203dacd2 196666 | 196656                     2
 4 00713653cf014c419427128400e7a5b0 73515                               1
 5 0096ddada2924340be01d83357e0d229 73719                               1
 6 00a2778c434d4b6f93524fa95da6a7ac 201119                              1
 7 00a4e4e6d6654b7f81d4b9be65008dc4 100287                              1
 8 00b2d3dd12b94c419ed9c851d601685f 198672                              1
 9 00be9daaa12c475e84571cd4124a086d 199320 | 199340                     2
10 00f68e50532546bf81ca2a6d2681fbef 73002 | 196228                      2
# … with 3,171 more rows


In [52]:
%%R
#join professionals_all_tags with prof_groups_df and prof_schools_df to combine information from group memberships and school memberships on professionals
prof_tags_groups_df <- left_join(prof_all_tags_df, prof_groups_df, by = c("professionals_id"="group_memberships_user_id"))  
prof_tags_groups_df <- left_join(prof_tags_groups_df, prof_schools_df, by = c("professionals_id"="school_memberships_user_id"))  
print(as_tibble(prof_tags_groups_df))

# A tibble: 28,152 × 12
   professionals_id           professionals_l… professionals_i… professional_in…
   <chr>                      <chr>            <chr>            <chr>           
 1 9ced4ce7519049c0944147afb… ""               ""               ""              
 2 f718dcf6d2ec4cb0a52a9db59… ""               ""               ""              
 3 0c673e046d824ec0ad0ebe012… "New York, New … ""               ""              
 4 977428d851b24183b223be0eb… "Boston, Massac… ""               ""              
 5 e2d57e5041a44f489288397c9… ""               ""               ""              
 6 c9bfa93898594cbbace436dec… ""               ""               ""              
 7 ed85488fb5e941eaa97014137… ""               ""               ""              
 8 102fb92c28034ad988b593d01… ""               ""               ""              
 9 5a4a16842ec64430ac3f916aa… ""               ""               ""              
10 81999d5ad93549dab55636a54… ""               ""               ""              
# … 

### Exploring emails and matches

In [53]:
#convert R datafram to python dataframe
%R -o emails_df 
display(HTML(emails_df.head().to_html()))

,emails_id,emails_recipient_id,emails_date_sent,emails_frequency_level
1,2337714,0c673e046d824ec0ad0ebe012a0673e4,2018-12-07 01:05:40 UTC+0000,email_notification_daily
2,2336077,0c673e046d824ec0ad0ebe012a0673e4,2018-12-06 01:14:15 UTC+0000,email_notification_daily
3,2314660,0c673e046d824ec0ad0ebe012a0673e4,2018-11-17 00:38:27 UTC+0000,email_notification_daily
4,2312639,0c673e046d824ec0ad0ebe012a0673e4,2018-11-16 00:32:19 UTC+0000,email_notification_daily
5,2299700,0c673e046d824ec0ad0ebe012a0673e4,2018-11-08 00:16:40 UTC+0000,email_notification_daily


In [54]:
#convert R datafram to python dataframe
%R -o matches_df 
display(HTML(matches_df.head().to_html()))

,matches_email_id,matches_question_id
1,1721939,332a511f1569444485cf7a7a556a5e54
2,1665388,332a511f1569444485cf7a7a556a5e54
3,1636634,332a511f1569444485cf7a7a556a5e54
4,1635498,332a511f1569444485cf7a7a556a5e54
5,1620298,332a511f1569444485cf7a7a556a5e54


In [55]:
%%R
print(dim(emails_df))
print(dim(matches_df))

[1] 1850101       4
[1] 4316275       2


In [56]:
%%R
#find duplicates in email ids and question ids
print(as_tibble(matches_df[duplicated(matches_df$matches_email_id), ]))
print(as_tibble(matches_df[duplicated(matches_df$matches_question_id), ]))

# A tibble: 2,522,190 × 2
   matches_email_id matches_question_id             
              <int> <chr>                           
 1          1426259 be3c5edfdb07423e955e9b2d7f186bce
 2          1303875 5cf8f5929e1c4357af4b467d4537dcef
 3          1687960 f156171f9c4944de99c1ab974cb7558d
 4          1550190 f156171f9c4944de99c1ab974cb7558d
 5          2018652 c1b82814b0274452873428b9db923939
 6          1644950 c1b82814b0274452873428b9db923939
 7          2205401 f05b579f2cd3432898f3ba54feee86b6
 8          1937172 f05b579f2cd3432898f3ba54feee86b6
 9          2018671 f05b579f2cd3432898f3ba54feee86b6
10          1847025 f05b579f2cd3432898f3ba54feee86b6
# … with 2,522,180 more rows
# A tibble: 4,294,381 × 2
   matches_email_id matches_question_id             
              <int> <chr>                           
 1          1665388 332a511f1569444485cf7a7a556a5e54
 2          1636634 332a511f1569444485cf7a7a556a5e54
 3          1635498 332a511f1569444485cf7a7a556a5e54
 4          162029

An email can be associated to multiple questions and multiple emails are sent to different professionals about the same question

In [57]:
%%R
#group matches_email_id to count of questions an email is related to
email_ques_df <- matches_df %>% group_by(matches_email_id) %>%
                  summarise(question_count = n_distinct(matches_question_id,na.rm = TRUE))
print(as_tibble(email_ques_df))

# A tibble: 1,794,085 × 2
   matches_email_id question_count
              <int>          <int>
 1              106              1
 2              107              1
 3              108              1
 4              110              1
 5              277              1
 6              279              1
 7              280              1
 8              283              1
 9              284              1
10              285              1
# … with 1,794,075 more rows


In [58]:
%%R
#extract date from emails date sent
emails_df <- emails_df %>% 
  mutate(date_sent = as.Date(emails_date_sent, format = "%Y-%m-%d"))
print(as_tibble(emails_df))

# A tibble: 1,850,101 × 5
   emails_id emails_recipient_id    emails_date_sent emails_frequenc… date_sent 
       <int> <chr>                  <chr>            <chr>            <date>    
 1   2337714 0c673e046d824ec0ad0eb… 2018-12-07 01:0… email_notificat… 2018-12-07
 2   2336077 0c673e046d824ec0ad0eb… 2018-12-06 01:1… email_notificat… 2018-12-06
 3   2314660 0c673e046d824ec0ad0eb… 2018-11-17 00:3… email_notificat… 2018-11-17
 4   2312639 0c673e046d824ec0ad0eb… 2018-11-16 00:3… email_notificat… 2018-11-16
 5   2299700 0c673e046d824ec0ad0eb… 2018-11-08 00:1… email_notificat… 2018-11-08
 6   2288533 0c673e046d824ec0ad0eb… 2018-11-02 23:0… email_notificat… 2018-11-02
 7   2280818 0c673e046d824ec0ad0eb… 2018-10-30 00:5… email_notificat… 2018-10-30
 8   2270520 0c673e046d824ec0ad0eb… 2018-10-25 23:4… email_notificat… 2018-10-25
 9   2269277 0c673e046d824ec0ad0eb… 2018-10-25 02:4… email_notificat… 2018-10-25
10   2267396 0c673e046d824ec0ad0eb… 2018-10-24 04:4… email_notificat… 2018-10-24
# 

In [59]:
%%R
unique(emails_df$emails_frequency_level)

[1] "email_notification_daily"     "email_notification_immediate"
[3] "email_notification_weekly"   


In [60]:
%%R
print(dim(emails_df))
print(dim(email_ques_df))

[1] 1850101       5
[1] 1794085       2


In [61]:
%%R
#join emails_df with email_ques_df
email_ques_df <- left_join(emails_df, email_ques_df, by = c("emails_id"="matches_email_id"))  
print(as_tibble(email_ques_df))

# A tibble: 1,850,101 × 6
   emails_id emails_recipient_id    emails_date_sent emails_frequenc… date_sent 
       <int> <chr>                  <chr>            <chr>            <date>    
 1   2337714 0c673e046d824ec0ad0eb… 2018-12-07 01:0… email_notificat… 2018-12-07
 2   2336077 0c673e046d824ec0ad0eb… 2018-12-06 01:1… email_notificat… 2018-12-06
 3   2314660 0c673e046d824ec0ad0eb… 2018-11-17 00:3… email_notificat… 2018-11-17
 4   2312639 0c673e046d824ec0ad0eb… 2018-11-16 00:3… email_notificat… 2018-11-16
 5   2299700 0c673e046d824ec0ad0eb… 2018-11-08 00:1… email_notificat… 2018-11-08
 6   2288533 0c673e046d824ec0ad0eb… 2018-11-02 23:0… email_notificat… 2018-11-02
 7   2280818 0c673e046d824ec0ad0eb… 2018-10-30 00:5… email_notificat… 2018-10-30
 8   2270520 0c673e046d824ec0ad0eb… 2018-10-25 23:4… email_notificat… 2018-10-25
 9   2269277 0c673e046d824ec0ad0eb… 2018-10-25 02:4… email_notificat… 2018-10-25
10   2267396 0c673e046d824ec0ad0eb… 2018-10-24 04:4… email_notificat… 2018-10-24
# 

In [62]:
%%R
#group by professional id(emails_recipient_id), date and emails_frequency and summarize email counts and question counts
email_prof_ques_df <- email_ques_df %>% group_by(emails_recipient_id, date_sent, emails_frequency_level) %>%
                  summarise(email_count = n_distinct(emails_id,na.rm = TRUE), question_count = sum(question_count,na.rm = TRUE) ) %>%
                  arrange(emails_recipient_id, emails_frequency_level,  date_sent) 
print(as_tibble(email_prof_ques_df))

`summarise()` has grouped output by 'emails_recipient_id', 'date_sent'. You can
override using the `.groups` argument.
# A tibble: 1,535,527 × 5
   emails_recipient_id    date_sent  emails_frequenc… email_count question_count
   <chr>                  <date>     <chr>                  <int>          <int>
 1 00009a0f9bda43eba4710… 2016-05-24 email_notificat…           1              1
 2 00009a0f9bda43eba4710… 2016-06-10 email_notificat…           1              1
 3 00009a0f9bda43eba4710… 2016-09-30 email_notificat…           1              2
 4 00009a0f9bda43eba4710… 2017-03-08 email_notificat…           1              1
 5 00009a0f9bda43eba4710… 2017-03-22 email_notificat…           1              1
 6 00009a0f9bda43eba4710… 2017-05-07 email_notificat…           1              1
 7 00009a0f9bda43eba4710… 2017-09-01 email_notificat…           1              1
 8 00009a0f9bda43eba4710… 2017-11-28 email_notificat…           1              2
 9 00009a0f9bda43eba4710… 2018-02-11 email_no

In [63]:
%%R
#separate the email notification on frequencies into different dataframes to summarise eve further
email_immediate_df <- email_prof_ques_df %>%
                  filter(emails_frequency_level == 'email_notification_immediate')
email_daily_df <- email_prof_ques_df %>%
                  filter(emails_frequency_level == 'email_notification_daily')
email_weekly_df <- email_prof_ques_df %>%
                  filter(emails_frequency_level == 'email_notification_weekly')

#group by to get mean counts of emails and questions
email_immediate_df <- email_immediate_df %>% group_by(emails_recipient_id) %>%
                  summarise(mean_immediate_emails = mean(email_count,na.rm = TRUE), mean_immediate_questions = mean(question_count,na.rm = TRUE) )
email_daily_df <- email_daily_df %>% group_by(emails_recipient_id) %>%
                  summarise(mean_daily_emails = mean(email_count,na.rm = TRUE), mean_daily_questions = mean(question_count,na.rm = TRUE) )
email_weekly_df <- email_weekly_df %>% group_by(emails_recipient_id) %>%
                  summarise(mean_weekly_emails = mean(email_count,na.rm = TRUE), mean_weekly_questions = mean(question_count,na.rm = TRUE) ) 

print(dim(email_immediate_df))
print(dim(email_daily_df))
print(dim(email_weekly_df))                                                    

[1] 4604    3
[1] 21014     3
[1] 1012    3


In [64]:
%%R
#join prof_tags_groups_df with email_immediate_df, email_daily_df and email_weekly_df to combine information about email notification frequency
prof_tags_groups_df <- left_join(prof_tags_groups_df, email_immediate_df, by = c("professionals_id"="emails_recipient_id"))  
prof_tags_groups_df <- left_join(prof_tags_groups_df, email_daily_df, by = c("professionals_id"="emails_recipient_id"))  
prof_tags_groups_df <- left_join(prof_tags_groups_df, email_weekly_df, by = c("professionals_id"="emails_recipient_id"))  
print(as_tibble(prof_tags_groups_df))

# A tibble: 28,152 × 18
   professionals_id           professionals_l… professionals_i… professional_in…
   <chr>                      <chr>            <chr>            <chr>           
 1 9ced4ce7519049c0944147afb… ""               ""               ""              
 2 f718dcf6d2ec4cb0a52a9db59… ""               ""               ""              
 3 0c673e046d824ec0ad0ebe012… "New York, New … ""               ""              
 4 977428d851b24183b223be0eb… "Boston, Massac… ""               ""              
 5 e2d57e5041a44f489288397c9… ""               ""               ""              
 6 c9bfa93898594cbbace436dec… ""               ""               ""              
 7 ed85488fb5e941eaa97014137… ""               ""               ""              
 8 102fb92c28034ad988b593d01… ""               ""               ""              
 9 5a4a16842ec64430ac3f916aa… ""               ""               ""              
10 81999d5ad93549dab55636a54… ""               ""               ""              
# … 

In [65]:
#convert R datafram to python dataframe
%R -o emails_df 
display(HTML(emails_df.head().to_html()))

,emails_id,emails_recipient_id,emails_date_sent,emails_frequency_level,date_sent
1,2337714,0c673e046d824ec0ad0ebe012a0673e4,2018-12-07 01:05:40 UTC+0000,email_notification_daily,17872.0
2,2336077,0c673e046d824ec0ad0ebe012a0673e4,2018-12-06 01:14:15 UTC+0000,email_notification_daily,17871.0
3,2314660,0c673e046d824ec0ad0ebe012a0673e4,2018-11-17 00:38:27 UTC+0000,email_notification_daily,17852.0
4,2312639,0c673e046d824ec0ad0ebe012a0673e4,2018-11-16 00:32:19 UTC+0000,email_notification_daily,17851.0
5,2299700,0c673e046d824ec0ad0ebe012a0673e4,2018-11-08 00:16:40 UTC+0000,email_notification_daily,17843.0


In [66]:
#convert R datafram to python dataframe
%R -o matches_df 
display(HTML(matches_df.head().to_html()))

,matches_email_id,matches_question_id
1,1721939,332a511f1569444485cf7a7a556a5e54
2,1665388,332a511f1569444485cf7a7a556a5e54
3,1636634,332a511f1569444485cf7a7a556a5e54
4,1635498,332a511f1569444485cf7a7a556a5e54
5,1620298,332a511f1569444485cf7a7a556a5e54


In [67]:
%%R
#create a join with matches and emails again, this time without summarizing on questions id
match_emails_df <- left_join(matches_df, emails_df, by = c("matches_email_id"="emails_id"))  
# arrange emails by question id,  emails_date_sent
match_emails_df <- match_emails_df %>% arrange(matches_question_id, emails_date_sent)
print(as_tibble(match_emails_df))

# A tibble: 4,316,275 × 6
   matches_email_id matches_question_id        emails_recipien… emails_date_sent
              <int> <chr>                      <chr>            <chr>           
 1          1523076 0003e7bf48f24b5c985f8fce9… fc83cccd92b54c2… 2018-01-22 17:4…
 2          1523075 0003e7bf48f24b5c985f8fce9… d08a870fdd7f4bf… 2018-01-22 17:4…
 3          1523079 0003e7bf48f24b5c985f8fce9… 992152c9327c415… 2018-01-22 17:4…
 4          1523078 0003e7bf48f24b5c985f8fce9… 4dba67cd990a4b5… 2018-01-22 17:4…
 5          1523077 0003e7bf48f24b5c985f8fce9… 41fe4234f2b04ad… 2018-01-22 17:4…
 6          1523082 0003e7bf48f24b5c985f8fce9… d67ce9308709451… 2018-01-22 17:4…
 7          1523081 0003e7bf48f24b5c985f8fce9… 7963583fd8f947c… 2018-01-22 17:4…
 8          1523080 0003e7bf48f24b5c985f8fce9… 0690fe8bea9d456… 2018-01-22 17:4…
 9          1523085 0003e7bf48f24b5c985f8fce9… 73cbd4b0241f4e9… 2018-01-22 17:4…
10          1523084 0003e7bf48f24b5c985f8fce9… a082cb6441ce48f… 2018-01-22 17:4…
# 

In [68]:
%%R
#group by question id and get the frequencies of emails sent irrespective of frequency level
ques_emails_df <- match_emails_df %>% group_by(matches_question_id)%>%
                  summarise(emails_sent_count = n_distinct(matches_email_id,na.rm = TRUE), professionals_notified_count=n_distinct(emails_recipient_id,na.rm = TRUE), first_email_sent = first(emails_date_sent), last_email_sent =  last(emails_date_sent))
print(as_tibble(ques_emails_df))

# A tibble: 21,894 × 5
   matches_question_id        emails_sent_cou… professionals_n… first_email_sent
   <chr>                                 <int>            <int> <chr>           
 1 0003e7bf48f24b5c985f8fce9…             1224             1224 2018-01-22 17:4…
 2 0006609dd4da40dcaa5a83e04…              277              277 2018-04-23 18:1…
 3 000af224bc2f4e94a19f8b62b…               48               48 2017-04-08 23:0…
 4 000b30fb534b41f7b716fa9eb…              129              129 2016-05-19 18:0…
 5 0018752e44b44e26bb74a0a43…               60               60 2018-01-17 03:5…
 6 00269cdc24fd4a3c93aad4856…               39               39 2018-02-05 23:5…
 7 002b07f6281a407ca49ee8b35…              113              113 2014-11-07 23:2…
 8 002c5689a0b642a3940f99b68…               13               13 2016-05-18 01:1…
 9 0031077b31454fadae1754e87…               39               39 2016-03-10 13:2…
10 00361e5dda874e65abaeeceb1…              110              110 2018-10-24 00:3…
# … w

In [69]:
%%R
#check if emails sent and professionals notified are different
ques_emails_df %>%
  filter(emails_sent_count != professionals_notified_count)

# A tibble: 854 × 5
   matches_question_id        emails_sent_cou… professionals_n… first_email_sent
   <chr>                                 <int>            <int> <chr>           
 1 00bb1d71eefa4ebfae1890bc7…              216              162 2016-05-23 16:1…
 2 00d85bcc5a26439d9a849d99f…              281              280 2016-05-10 16:1…
 3 00e934af910e4c92a530e1697…               52               51 2014-04-08 12:3…
 4 010b3de7532946148289efa07…              205              204 2016-05-24 23:0…
 5 0263ec137c424d8ca48edc388…              276              275 2016-05-22 01:2…
 6 027722cb9e284abda9b3f05dc…               51               50 2016-05-24 22:3…
 7 027eb836e57449b3b968fcfad…              228              227 2016-05-18 20:1…
 8 027ecc7704c346f598a0b6584…               27               26 2014-04-07 13:4…
 9 0284380052104b3aba7e48554…              369              368 2016-03-01 16:4…
10 0285ed4ea0a348f9900f35267…               79               59 2016-05-23 02:2…
# … with

In [70]:
%%R
print(as_tibble(ques_score_tag_df))

# A tibble: 23,931 × 8
   questions_id questions_autho… questions_date_… questions_title questions_body
   <chr>        <chr>            <chr>            <chr>           <chr>         
 1 332a511f156… 8f6f374ffd834d2… 2016-04-26 11:1… Teacher   care… "What  is  a …
 2 eb80205482e… acccbda28edd436… 2016-05-20 16:4… I want to beco… "I am Priyank…
 3 4ec31632938… f2c179a563024cc… 2017-02-08 19:1… Will going abr… "I'm planning…
 4 2f6a9a99d9b… 2c30ffba444e40e… 2017-09-01 14:0… To become a sp… "i hear busin…
 5 5af8880460c… aa9eb1a2ab184eb… 2017-09-01 02:3… Are there any … "I'm trying t…
 6 7c336403258… d1e4587c0e784c6… 2017-03-01 04:2… How many years… "To be an eng…
 7 be3c5edfdb0… 71b4554d4a82425… 2017-09-01 04:5… I want to beco… "I am a music…
 8 0f1d6a4f276… 585ac233015447c… 2016-05-19 22:1… what kind of  … "I like socce…
 9 d4999cdc470… 654e1d6fd5b9472… 2017-08-31 19:2… What are the c… "I'm asking b…
10 e214acfbe66… 16908136951a48e… 2012-09-09 05:3… what does it t… "I am a sopho…
# … w

In [71]:
%%R
#join ques_score_tag_df with ques_emails_df
ques_score_tag_df <- left_join(ques_score_tag_df, ques_emails_df, by = c("questions_id"="matches_question_id"))  
print(as_tibble(ques_score_tag_df))

# A tibble: 23,931 × 12
   questions_id questions_autho… questions_date_… questions_title questions_body
   <chr>        <chr>            <chr>            <chr>           <chr>         
 1 332a511f156… 8f6f374ffd834d2… 2016-04-26 11:1… Teacher   care… "What  is  a …
 2 eb80205482e… acccbda28edd436… 2016-05-20 16:4… I want to beco… "I am Priyank…
 3 4ec31632938… f2c179a563024cc… 2017-02-08 19:1… Will going abr… "I'm planning…
 4 2f6a9a99d9b… 2c30ffba444e40e… 2017-09-01 14:0… To become a sp… "i hear busin…
 5 5af8880460c… aa9eb1a2ab184eb… 2017-09-01 02:3… Are there any … "I'm trying t…
 6 7c336403258… d1e4587c0e784c6… 2017-03-01 04:2… How many years… "To be an eng…
 7 be3c5edfdb0… 71b4554d4a82425… 2017-09-01 04:5… I want to beco… "I am a music…
 8 0f1d6a4f276… 585ac233015447c… 2016-05-19 22:1… what kind of  … "I like socce…
 9 d4999cdc470… 654e1d6fd5b9472… 2017-08-31 19:2… What are the c… "I'm asking b…
10 e214acfbe66… 16908136951a48e… 2012-09-09 05:3… what does it t… "I am a sopho…
# … 

### Exploring students, answers, answer_scores and comments

In [72]:
#convert R datafram to python dataframe
%R -o students_df 
display(HTML(students_df.head().to_html()))

,students_id,students_location,students_date_joined
1,12a89e96755a4dba83ff03e03043d9c0,,2011-12-16 14:19:24 UTC+0000
2,e37a5990fe354c60be5e87376b08d5e3,,2011-12-27 03:02:44 UTC+0000
3,12b402cceeda43dcb6e12ef9f2d221ea,,2012-01-01 05:00:00 UTC+0000
4,a0f431fc79794edcb104f68ce55ab897,,2012-01-01 05:00:00 UTC+0000
5,23aea4702d804bd88d1e9fb28074a1b4,,2012-01-01 05:00:00 UTC+0000


In [73]:
#convert R datafram to python dataframe
%R -o answers_df 
display(HTML(answers_df.head().to_html()))

,answers_id,answers_author_id,answers_question_id,answers_date_added,answers_body
1,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,2016-04-29 19:40:14 UTC+0000,<p>Hi!</p>\n<p>You are asking a very interesting question. I am giving you two sites that will give you some of an explanation that may answer your question.</p>\n<p>http://mathforum.org/dr.math/faq/faq.why.math.html</p>\n<p>http://www.mathworksheetscenter.com/mathtips/mathissoimportant.html</p>\n<p>Let me know if this helps</p>
2,ada720538c014e9b8a6dceed09385ee3,2aa47af241bf42a4b874c453f0381bd4,eb80205482e4424cad8f16bc25aa2d9c,2018-05-01 14:19:08 UTC+0000,"<p>Hi. I joined the Army after I attended college and received a Bachelor's Degree in Criminal Justice. Commissioned officers enter the Military with a four year degree or receive officer training after joining and complete a tour. You can prepare yourself by taking Reserve Officer Training Corps (ROTC) while in high school or a university. You can also attend Officer Candidate School (OCS) after graduating from college or become commissioned by earning a professional degree. If you decide to earn a degree, think about what you would like to do while in the military. The experience in the military was very rewarding. I wish you well and much success in your future. </p>"
3,eaa66ef919bc408ab5296237440e323f,cbd8f30613a849bf918aed5c010340be,eb80205482e4424cad8f16bc25aa2d9c,2018-05-02 02:41:02 UTC+0000,"<p>Dear Priyanka,</p><p>Greetings! I have answered this question to Eshwari few days ago. I am going to reproduce that answer with bit of modifications as required for better clarity. </p><p><br></p><p>From your background, I could make out that you are from Bangalore and a student of 10th standard but your location is Rhode Island, USA. So I am not very clear as to which Army you wish to join ? Is it the US Army or the Indian Army? It is important to know that very few foreign nationals can join Indian Army (like Nepali citizens etc.). So you have to ascertain for yourself as to which citizenship you hold and accordingly you can join armed forces of the respective countries. Both are highly professional armies and respected a lot. I shall answer the modalities about the Indian Army, assuming that is the natural choice.</p><p><br></p><p>So, to answer your query, there are following options for you to become army officer:</p><ol><li> Do your B.Sc and that will be good as you have wider choices including flying branches in air force and executive branches in Navy. Similarly BA/B.Com if you are not looking for flying or executive branches. </li><li> Should you wish to be doctor in Army then you can appear for entrance Test for AFMC, Pune or Army Dental College after class 12th with PCMB. However, you can do the same by attending Medical colleges from civil institutes in India/abroad. </li><li> You can also join Military Nursing Services after B.Sc or Diploma in Nursing but majority of them are trained within army organisations (after 12th only with PCMB). </li><li> You can join after doing your graduation in Engineering (Civil, Mech, Electrical, Electronics or Computer Sciences) or Masters in Physics with Electronics or Computers for engineering branches.</li><li>You can also join in Education Corps by doing M.Sc/MA/M.Com with or without B.Ed or in Legal services by doing your Law Degree. </li></ol><p><br></p><p>After your basic education, application and selection test processes, one has to clear SSB (Services Selection Board) which is a very stringent 5 days test (compulsory for all including Women officers except medical professionals). It is one of the best methods of test I know to assess the suitability for military services (as officers), a proven method of testing of one's psyche, leadership qualities which takes all the aspects of your personality into account and decide. Although, success rate is pretty low but don't get disheartened and demotivated. Key to succes

In [74]:
#convert R datafram to python dataframe
%R -o answer_scores_df 
display(HTML(answer_scores_df.head().to_html()))

,id,score
1,7b2bb0fc0d384e298cffa6afde9cf6ab,1
2,7640a6e5d5224c8681cc58de860858f4,5
3,3ce32e236fa9435183b2180fb213375c,2
4,fa30fe4c016043e382c441a7ef743bfb,0
5,71229eb293314c8a9e545057ecc32c93,2


In [75]:
#convert R datafram to python dataframe
%R -o comments_df 
display(HTML(comments_df.head().to_html()))

,comments_id,comments_author_id,comments_parent_content_id,comments_date_added,comments_body
1,f30250d3c2ca489db1afa9b95d481e08,9fc88a7c3323466dbb35798264c7d497,b476f9c6d9cd4c50a7bacdd90edd015a,2019-01-31 23:39:40 UTC+0000,"First, you speak to recruiters. They are trained and knowledgable on all the requirements for each branch of service and can do the research for you on the specific job that you are looking at. \n\nAdditionally, birds of a feather do flock together. In high school, you can join groups like the Civil Air Patrol. The Civil Air Patrol is the auxiliary of the Air Force and many former and retired Air Force member within their ranks."
2,ca9bfc4ba9464ea383a8b080301ad72c,de2415064b9b445c8717425ed70fd99a,ef4b6ae24d1f4c3b977731e8189c7fd7,2019-01-31 20:30:47 UTC+0000,Most large universities offer study abroad programs. The study abroad programs are found on the schools website. You may have to click into the Undergraduate or Graduate links. I also recommend reaching out to the university counselors. The have wealth of information and details that may not be on the website. \n\nMy son will be studying in France and Sweden next year through DePaul University. The University of Southern California has a very strong international program in which most students participate. It's a great opportunity to learn the culture of other countries while in school.
3,c354f6e33956499aa8b03798a60e9386,6ed20605002a42b0b8e3d6ac97c50c7f,ca7a9d7a95df471c816db82ee758f57d,2019-01-31 18:44:04 UTC+0000,"First, I want to put you at ease that the opposite can happen. My dormmate that I was paired with my freshman year of college turned into one of my very best friends. Second, we lived with another girl during our sophomore year; a good friend of ours that lived next to us the previous year. To us, she was the annoying dormmate that you are mentioning and we already knew her! Sometimes it takes living with someone to learn their annoying habits. We could have swapped if someone else agreed to swap with us. Instead we tried to talk to her about it. When it did not get better, my other dormmate and I strategized how we could avoid the annoying habits. For instance, we agreed that the habits were most annoying when we were trying to study so we studied elsewhere."
4,73a6223948714c5da6231937157e4cb7,d02f6d9faac24997a7003a59e5f34bd3,c7a88aa76f5f49b0830bfeb46ba17e4d,2019-01-31 17:53:28 UTC+0000,"Your question submission was great! I just wanted to point out that if you break your original question into separate and slightly more specific points (like ""In addition to the LSAT what is the Law School acceptance process like..."" you might also get some great Advice from our professionals. General questions are always welcome too, just make sure to leave some detail (like you did) of why you are interested etc. Welcome to the CareerVillage community!"
5,55a89a9061d44dd19569c45f90a22779,e78f75c543e84e1c94da1801d8560f65,c7a88aa76f5f49b0830bfeb46ba17e4d,2019-01-31 14:51:53 UTC+0000,Thank you. I'm new to this site. I'm sorry if what I put out there is spam. I will for sure read through those forums.


In [76]:
%%R
#find duplicates in question id in questions and questions_score
print(as_tibble(answers_df[duplicated(answers_df$answers_id), ]))
print(as_tibble(answer_scores_df[duplicated(answer_scores_df$id), ]))
print(dim(answers_df))
print(dim(answer_scores_df))
ans_anti_join_df <- anti_join(answer_scores_df, answers_df, by = c("id"="answers_id"))
print(as_tibble(ans_anti_join_df)) 
ans_anti_join2_df <- anti_join(answers_df, answer_scores_df, by = c("answers_id"="id"))
print(as_tibble(ans_anti_join2_df)) 

# A tibble: 0 × 5
# … with 5 variables: answers_id <chr>, answers_author_id <chr>,
#   answers_question_id <chr>, answers_date_added <chr>, answers_body <chr>
# A tibble: 0 × 2
# … with 2 variables: id <chr>, score <int>
[1] 51123     5
[1] 51138     2
# A tibble: 31 × 2
   id                               score
   <chr>                            <int>
 1 815e25efd9c746a7beca5a28b7a1fc0f     1
 2 8cd34d4d62b04509910a05a790fe930d     0
 3 91f221806cfb4ea986eece7b3e94a236     0
 4 5a15a37f4940452296e487622c59b786     0
 5 fd8bdbb3029f4d51ad73ae249f2ffbde     0
 6 cb46e274ab3d491a89311d8c27c4c7c4     0
 7 2b1baf69bef742dfa1e247c5350afd0d     2
 8 2da9a49ab7ec4b46b4211b7dbfa61497     2
 9 5c081a4ef54641a6a3664cc02ccc55d0     1
10 7452fbb15f4d4538ad11b2a6e8668b6e     0
# … with 21 more rows
# A tibble: 16 × 5
   answers_id    answers_author_… answers_questio… answers_date_ad… answers_body
   <chr>         <chr>            <chr>            <chr>            <chr>       
 1 c28bf10c8e0d… 3465

Relationship between answers and answers scores is One to One. Some answers do not have a score and some scores are not associated to the ids in answers

In [77]:
%%R
#join answers with answer score
answers_df <- left_join(answers_df, answer_scores_df, by = c("answers_id"="id"))  
print(as_tibble(answers_df))

# A tibble: 51,123 × 6
   answers_id    answers_author_… answers_questio… answers_date_ad… answers_body
   <chr>         <chr>            <chr>            <chr>            <chr>       
 1 4e5f01128cae… 36ff3b3666df400… 332a511f1569444… 2016-04-29 19:4… "<p>Hi!</p>…
 2 ada720538c01… 2aa47af241bf42a… eb80205482e4424… 2018-05-01 14:1… "<p>Hi. I j…
 3 eaa66ef919bc… cbd8f30613a849b… eb80205482e4424… 2018-05-02 02:4… "<p>Dear Pr…
 4 1a6b3749d391… 7e72a630c303442… 4ec31632938a40b… 2017-05-10 19:0… "<p>I work …
 5 5229c5140004… 17802d94699140b… 2f6a9a99d9b24e5… 2017-10-13 22:0… "I agree wi…
 6 5f62fadae807… b03c3872daeb4a5… 2f6a9a99d9b24e5… 2017-10-12 16:0… "Networking…
 7 1d804b3b9e76… f6c89fde797d459… 5af8880460c141d… 2017-09-29 18:5… "https://ww…
 8 08d3cf6fa205… 5bc2db4d58584f9… 5af8880460c141d… 2018-06-08 02:3… "<p>Hi Joce…
 9 9d33d099a03c… 8136c8653d3a489… 7c336403258f4da… 2017-03-01 16:5… "<p>For the…
10 228e5feefb4e… 35c8d979b566478… be3c5edfdb07423… 2017-10-13 18:0… "Of course!…
# … w

In [78]:
%%R
# get frequencies of questions posted by students
student_ques_df <- ques_score_tag_df %>% select(questions_id,questions_author_id,questions_date_added) %>%
                  arrange(questions_author_id,questions_date_added)
student_ques_df <- student_ques_df %>% group_by(questions_author_id) %>%
                  summarise(total_question_count = n_distinct(questions_id,na.rm = TRUE), first_question_posted = first(questions_date_added), last_question_posted =  last(questions_date_added))
print(as_tibble(student_ques_df))

# A tibble: 12,329 × 4
   questions_author_id        total_question_… first_question_… last_question_p…
   <chr>                                 <int> <chr>            <chr>           
 1 0001a66883f74e8d91884fc6e…                2 2018-08-21 00:2… 2018-08-21 00:2…
 2 0004592176864233990d94986…                1 2016-03-09 17:1… 2016-03-09 17:1…
 3 001bd6f7f1ac4897b0b35dc66…                4 2018-08-29 04:4… 2018-09-28 00:5…
 4 001d1629d09d441a8f09e3bfa…                2 2016-05-26 00:0… 2016-05-26 00:2…
 5 001df2a2cfb74ef78f3053dde…                2 2015-06-26 22:4… 2015-07-03 02:2…
 6 001f3b39d8884a41ab7e71789…                2 2018-01-17 18:5… 2018-01-17 18:5…
 7 0029e4c8e1c14b12be6d1d9a6…                2 2014-03-05 21:4… 2014-03-14 19:4…
 8 003a1ef91d0a45f6a61ca56ff…                2 2017-05-15 19:0… 2017-05-15 19:1…
 9 003af75eba604db9a7b72862f…                1 2015-01-30 04:5… 2015-01-30 04:5…
10 003b69a301354c3d9d23e0b8c…                2 2016-05-12 12:0… 2016-05-12 12:1…
# … w

In [79]:
%%R
# get frequencies of comments posted by students and professionals
all_comments_df <- comments_df %>% select(comments_id,comments_author_id,comments_date_added) %>%
                  arrange(comments_author_id,comments_date_added)
all_comments_df <- all_comments_df %>% group_by(comments_author_id) %>%
                  summarise(total_comment_count = n_distinct(comments_id,na.rm = TRUE), first_comment_posted = first(comments_date_added), last_comment_posted =  last(comments_date_added))
print(as_tibble(all_comments_df))

# A tibble: 4,231 × 4
   comments_author_id         total_comment_c… first_comment_p… last_comment_po…
   <chr>                                 <int> <chr>            <chr>           
 1 0017911d8762492785352f8cb…                1 2017-05-20 15:0… 2017-05-20 15:0…
 2 001f3b39d8884a41ab7e71789…                1 2018-02-11 18:5… 2018-02-11 18:5…
 3 0035e45e1f914bc9aa273b2ad…                1 2018-05-17 20:4… 2018-05-17 20:4…
 4 003664e8e06348959946fda7c…                2 2017-11-06 15:5… 2017-11-06 15:5…
 5 0036eaccbe364ba8bbf7658f1…                1 2018-05-11 17:0… 2018-05-11 17:0…
 6 003a1ef91d0a45f6a61ca56ff…                1 2017-05-15 19:1… 2017-05-15 19:1…
 7 004b221ad6534a0fac421356e…                1 2018-09-30 19:0… 2018-09-30 19:0…
 8 005d8187b0d149f2bcc6a6f0f…                1 2018-05-02 22:3… 2018-05-02 22:3…
 9 0063387ed69a4080a7a65b010…                1 2016-05-18 20:2… 2016-05-18 20:2…
10 006b57ed965c43ea981190483…                1 2016-11-24 02:5… 2016-11-24 02:5…
# … wi

In [80]:
%%R
#join students with student_ques_df and all_comments_df
students_df <- left_join(students_df, student_ques_df, by = c("students_id"="questions_author_id"))  
students_df <- left_join(students_df, all_comments_df, by = c("students_id"="comments_author_id"))  
print(as_tibble(students_df))

# A tibble: 30,971 × 9
   students_id                students_locati… students_date_j… total_question_…
   <chr>                      <chr>            <chr>                       <int>
 1 12a89e96755a4dba83ff03e03… ""               2011-12-16 14:1…                1
 2 e37a5990fe354c60be5e87376… ""               2011-12-27 03:0…               NA
 3 12b402cceeda43dcb6e12ef9f… ""               2012-01-01 05:0…               NA
 4 a0f431fc79794edcb104f68ce… ""               2012-01-01 05:0…               NA
 5 23aea4702d804bd88d1e9fb28… ""               2012-01-01 05:0…               NA
 6 18a8f9363cd24a37b690e1b20… ""               2012-01-01 05:0…               NA
 7 d21c67279ada49d1bcf66ad62… ""               2012-01-01 05:0…               NA
 8 433c0f5f90344453ba50ec8ae… ""               2012-01-01 05:0…               NA
 9 f0a7d23c3d374f8d9a1a5eb9b… ""               2012-01-01 05:0…               NA
10 26269f6765b74347b28155aea… ""               2012-01-01 05:0…               NA
# … w

In [81]:
%%R
print(as_tibble(students_df))

# A tibble: 30,971 × 9
   students_id                students_locati… students_date_j… total_question_…
   <chr>                      <chr>            <chr>                       <int>
 1 12a89e96755a4dba83ff03e03… ""               2011-12-16 14:1…                1
 2 e37a5990fe354c60be5e87376… ""               2011-12-27 03:0…               NA
 3 12b402cceeda43dcb6e12ef9f… ""               2012-01-01 05:0…               NA
 4 a0f431fc79794edcb104f68ce… ""               2012-01-01 05:0…               NA
 5 23aea4702d804bd88d1e9fb28… ""               2012-01-01 05:0…               NA
 6 18a8f9363cd24a37b690e1b20… ""               2012-01-01 05:0…               NA
 7 d21c67279ada49d1bcf66ad62… ""               2012-01-01 05:0…               NA
 8 433c0f5f90344453ba50ec8ae… ""               2012-01-01 05:0…               NA
 9 f0a7d23c3d374f8d9a1a5eb9b… ""               2012-01-01 05:0…               NA
10 26269f6765b74347b28155aea… ""               2012-01-01 05:0…               NA
# … w

In [82]:
%%R
# get frequencies of answers posted by professionals
prof_ques_answered_df <- answers_df %>% select(answers_id,answers_author_id,answers_question_id, answers_date_added) %>%
                  arrange(answers_author_id,answers_date_added)
prof_ques_answered_df <- prof_ques_answered_df %>% group_by(answers_author_id) %>%
                  summarise(total_questions_answered = n_distinct(answers_question_id,na.rm = TRUE), total_answers_posted = n_distinct(answers_id,na.rm = TRUE), first_answer_posted = first(answers_date_added), last_answer_posted =  last(answers_date_added))
print(as_tibble(prof_ques_answered_df))

# A tibble: 10,169 × 5
   answers_author_id          total_questions… total_answers_p… first_answer_po…
   <chr>                                 <int>            <int> <chr>           
 1 00009a0f9bda43eba47104e9a…                3                3 2016-03-14 17:3…
 2 000d4635e5da41e3bfd83677e…                3                3 2016-04-27 16:4…
 3 00271cc10e0245fba4a35e76e…               24               24 2018-09-12 16:5…
 4 003cc21be89d4e42bc4424131…                4                4 2017-10-19 18:1…
 5 0046ab8089c04b3a8df3f8c28…                5                5 2017-05-10 18:0…
 6 004cb439b2fb4abcbf823380a…                3                3 2018-08-21 23:2…
 7 0053becb71d94164b014a5a7d…                2                2 2016-11-10 18:5…
 8 005cbd7ae54949db98efaa356…                1                1 2018-09-26 00:5…
 9 00680f924e8f49d4962876df5…                3                3 2016-06-07 08:4…
10 007e521571a248378a7f335fc…                3                3 2016-10-27 13:2…
# … w

In [83]:
%%R
#join professionals with prof_ques_answered_df and all_comments_df
prof_tags_groups_df <- left_join(prof_tags_groups_df, prof_ques_answered_df, by = c("professionals_id"="answers_author_id"))  
prof_tags_groups_df <- left_join(prof_tags_groups_df, all_comments_df, by = c("professionals_id"="comments_author_id"))  
print(as_tibble(prof_tags_groups_df))

# A tibble: 28,152 × 25
   professionals_id           professionals_l… professionals_i… professional_in…
   <chr>                      <chr>            <chr>            <chr>           
 1 9ced4ce7519049c0944147afb… ""               ""               ""              
 2 f718dcf6d2ec4cb0a52a9db59… ""               ""               ""              
 3 0c673e046d824ec0ad0ebe012… "New York, New … ""               ""              
 4 977428d851b24183b223be0eb… "Boston, Massac… ""               ""              
 5 e2d57e5041a44f489288397c9… ""               ""               ""              
 6 c9bfa93898594cbbace436dec… ""               ""               ""              
 7 ed85488fb5e941eaa97014137… ""               ""               ""              
 8 102fb92c28034ad988b593d01… ""               ""               ""              
 9 5a4a16842ec64430ac3f916aa… ""               ""               ""              
10 81999d5ad93549dab55636a54… ""               ""               ""              
# … 

In [84]:
%%R
print(dim(answers_df))
print(dim(comments_df))

[1] 51123     6
[1] 14966     5


In [85]:
%%R
#combine comments and answers and group by answer id to get the comment count 
answer_comments_df <- left_join(answers_df, comments_df, by = c("answers_id"="comments_parent_content_id"))
answer_comments_df <- answer_comments_df %>% arrange(answers_question_id, answers_date_added, comments_date_added)
answer_comments_df <- answer_comments_df %>% group_by(answers_id) %>%
                                    summarise(total_comments = n_distinct(comments_id,na.rm = TRUE), first_comment_posted = first(comments_date_added), last_comment_posted =  last(comments_date_added))
answers_df <- left_join(answers_df, answer_comments_df, by = c("answers_id"="answers_id"))
print(as_tibble(answer_comments_df))
print(as_tibble(answers_df))

# A tibble: 51,123 × 4
   answers_id                   total_comments first_comment_p… last_comment_po…
   <chr>                                 <int> <chr>            <chr>           
 1 0000de0a3eba4db7bc0259dd915…              0 <NA>             <NA>            
 2 000117a4c2f74984b86b0808976…              0 <NA>             <NA>            
 3 0001ee43a054491585649a9dc7f…              0 <NA>             <NA>            
 4 00026eea175a4294b084deb9ed0…              0 <NA>             <NA>            
 5 000350c27a6a4b53b32e7f1c107…              0 <NA>             <NA>            
 6 0003bd9de5f74c93880afc5f5d4…              0 <NA>             <NA>            
 7 000535ea5b584a1195c1dfe34ec…              2 2017-09-11 04:2… 2017-09-11 06:5…
 8 00053c080c42477ebec318381e9…              0 <NA>             <NA>            
 9 0006238d9cf24881afbaf505aee…              0 <NA>             <NA>            
10 00077251e4554d1994d8b155f05…              0 <NA>             <NA>            
# … w

In [86]:
%%R
#combine answers and questions and group by question id to get the answer count 
question_answers_df <- right_join(questions_df, answers_df, by = c("questions_id"="answers_question_id"))
question_answers_df <- question_answers_df %>% arrange(questions_id, answers_date_added)
question_answers_df <- question_answers_df %>% group_by(questions_id) %>%
                                    summarise(total_answers = n_distinct(answers_id,na.rm = TRUE), first_answer_posted = first(answers_date_added), last_answer_posted =  last(answers_date_added))

#combine comments and questions and group by question id to get the comment count 
question_comments_df <- left_join(answers_df, comments_df, by = c("answers_id"="comments_parent_content_id"))
question_comments_df <- question_comments_df %>% arrange(answers_question_id, comments_date_added)
question_comments_df <- question_comments_df %>% group_by(answers_question_id) %>%
                                    summarise(total_comments = n_distinct(comments_id,na.rm = TRUE), first_comment_posted = first(comments_date_added), last_comment_posted =  last(comments_date_added))
ques_score_tag_df <- left_join(ques_score_tag_df, question_answers_df, by = c("questions_id"="questions_id"))
ques_score_tag_df <- left_join(ques_score_tag_df, question_comments_df, by = c("questions_id"="answers_question_id"))
print(as_tibble(ques_score_tag_df))

# A tibble: 23,931 × 18
   questions_id questions_autho… questions_date_… questions_title questions_body
   <chr>        <chr>            <chr>            <chr>           <chr>         
 1 332a511f156… 8f6f374ffd834d2… 2016-04-26 11:1… Teacher   care… "What  is  a …
 2 eb80205482e… acccbda28edd436… 2016-05-20 16:4… I want to beco… "I am Priyank…
 3 4ec31632938… f2c179a563024cc… 2017-02-08 19:1… Will going abr… "I'm planning…
 4 2f6a9a99d9b… 2c30ffba444e40e… 2017-09-01 14:0… To become a sp… "i hear busin…
 5 5af8880460c… aa9eb1a2ab184eb… 2017-09-01 02:3… Are there any … "I'm trying t…
 6 7c336403258… d1e4587c0e784c6… 2017-03-01 04:2… How many years… "To be an eng…
 7 be3c5edfdb0… 71b4554d4a82425… 2017-09-01 04:5… I want to beco… "I am a music…
 8 0f1d6a4f276… 585ac233015447c… 2016-05-19 22:1… what kind of  … "I like socce…
 9 d4999cdc470… 654e1d6fd5b9472… 2017-08-31 19:2… What are the c… "I'm asking b…
10 e214acfbe66… 16908136951a48e… 2012-09-09 05:3… what does it t… "I am a sopho…
# … 

In [87]:
%%R
#combine questions with student details
ques_score_tag_df <- left_join(ques_score_tag_df, students_df, by = c("questions_author_id"="students_id"))
#rename score column in ques_score_tag_df to "question_score"
names(ques_score_tag_df)[names(ques_score_tag_df) == 'score'] <- 'question_score'
print(as_tibble(ques_score_tag_df))

#combine answers with professionals details
answers_df <- left_join(answers_df, select(prof_tags_groups_df, c(professionals_id,professionals_location,professionals_industry,professionals_headline,professionals_date_joined)), by = c("answers_author_id"="professionals_id"))
#rename score column in answers_df to "answer_score"
names(answers_df)[names(answers_df) == 'score'] <- 'answer_score'
#combine answers with question details
answers_df <- left_join(answers_df, select(ques_score_tag_df,c(questions_id,questions_date_added,all_tags,tag_count)), by = c("answers_question_id"="questions_id"))
print(as_tibble(answers_df))


# A tibble: 23,931 × 26
   questions_id questions_autho… questions_date_… questions_title questions_body
   <chr>        <chr>            <chr>            <chr>           <chr>         
 1 332a511f156… 8f6f374ffd834d2… 2016-04-26 11:1… Teacher   care… "What  is  a …
 2 eb80205482e… acccbda28edd436… 2016-05-20 16:4… I want to beco… "I am Priyank…
 3 4ec31632938… f2c179a563024cc… 2017-02-08 19:1… Will going abr… "I'm planning…
 4 2f6a9a99d9b… 2c30ffba444e40e… 2017-09-01 14:0… To become a sp… "i hear busin…
 5 5af8880460c… aa9eb1a2ab184eb… 2017-09-01 02:3… Are there any … "I'm trying t…
 6 7c336403258… d1e4587c0e784c6… 2017-03-01 04:2… How many years… "To be an eng…
 7 be3c5edfdb0… 71b4554d4a82425… 2017-09-01 04:5… I want to beco… "I am a music…
 8 0f1d6a4f276… 585ac233015447c… 2016-05-19 22:1… what kind of  … "I like socce…
 9 d4999cdc470… 654e1d6fd5b9472… 2017-08-31 19:2… What are the c… "I'm asking b…
10 e214acfbe66… 16908136951a48e… 2012-09-09 05:3… what does it t… "I am a sopho…
# … 

## Save files for future use

In [91]:
%%R
#fix all null values
ques_score_tag_df <- ques_score_tag_df %>% mutate_if(is.integer, ~replace(., is.na(.), 0)) %>% mutate_if(is.numeric, ~replace(., is.na(.), 0))
prof_tags_groups_df <- prof_tags_groups_df %>% mutate_if(is.integer, ~replace(., is.na(.), 0)) %>% mutate_if(is.numeric, ~replace(., is.na(.), 0))
answers_df <- answers_df %>% mutate_if(is.integer, ~replace(., is.na(.), 0)) %>% mutate_if(is.numeric, ~replace(., is.na(.), 0))
tag_ques_prof_df <- tag_ques_prof_df %>% mutate_if(is.integer, ~replace(., is.na(.), 0)) %>% mutate_if(is.numeric, ~replace(., is.na(.), 0))

`mutate_if()` ignored the following grouping variables:
• Column `tags_tag_id`
`mutate_if()` ignored the following grouping variables:
• Column `tags_tag_id`


In [92]:
from google.colab import files

#convert R data fram to python dataframe
%R -o ques_score_tag_df
ques_score_tag_df.to_csv('questions.csv', encoding = 'utf-8-sig') 
files.download('questions.csv')

%R -o prof_tags_groups_df
prof_tags_groups_df.to_csv('professionals.csv', encoding = 'utf-8-sig') 
files.download('professionals.csv')

%R -o answers_df
answers_df.to_csv('answers.csv', encoding = 'utf-8-sig') 
files.download('answers.csv')

%R -o comments_df
comments_df.to_csv('comments.csv', encoding = 'utf-8-sig') 
files.download('comments.csv')

%R -o tag_ques_prof_df
tag_ques_prof_df.to_csv('tags.csv', encoding = 'utf-8-sig') 
files.download('tags.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Session Info

In [90]:
%%R
sessionInfo()

R version 4.2.0 (2022-04-22)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 18.04.5 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/openblas/liblapack.so.3

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] tools     stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] lubridate_1.8.0 plotly_4.10.0   forcats_0.5.1   stringr_1.4.0  
 [5] dplyr_1.0.9     purrr_0.3.4     readr_2.1.2     tidyr_1.2.0    
 [9] tibble_3.1.6    ggplot2_3.3.5   tidyverse_1.3.1

loaded via a namespace (and not attached):
 [1] cellranger_1.1.0